<img src="./files/refinitiv.png" width="20%" style="vertical-align: top;">\

---- 
This source code is provided under the Apache 2.0 license\
and is provided AS IS with no warranty or guarantee of fit for purpose.\
See the project's LICENSE.md for details.\
Copyright (C) 2022 Refinitiv. All rights reserved.
---- 

# Authentication Token Handling
This is not an independent example, this code is inteded to be imported for use by other modules

### Import Libraries and Define Application Constants

In [17]:
import requests, json, time

# Set Application Constants
RDP_AUTH_VERSION = "/v1"
RDP_CFS_VERSION = "/v1"
RDP_BASE_URL = "https://api.refinitiv.com"
CATEGORY_URL = "/auth/oauth2"
ENDPOINT_URL = "/token"
CLIENT_SECRET = ""
TOKEN_FILE = "token.txt"
SCOPE = "trapi"

### Set Valid Credentials 

Valid RDP credentials are required to proceed:
* USERNAME
* PASSWORD
* CLIENTID

To read one's valid credentials from a file (that can be shared by many code examples), leave below code as is.

To provide credentials in place:
* replace the commented credentials with one's valid assigned credentials
* comment the read from file step readCredsFromFile

In [18]:
USERNAME = "VALIDUSER"
PASSWORD = "VALIDPASSWORD"
CLIENT_ID = "SELFGENERATEDCLIENTID"

def readCredsFromFile(filePathName):
### Read valid credentials from file
    global USERNAME, PASSWORD, CLIENT_ID
    credFile = open(filePathName,"r")    # one per line
                                                #--- RDP MACHINE ID---
                                                #--- LONG PASSWORD---
                                                #--- GENERATED CLIENT ID---

    USERNAME = credFile.readline().rstrip('\n')
    PASSWORD = credFile.readline().rstrip('\n')
    CLIENT_ID = credFile.readline().rstrip('\n')

    credFile.close()

readCredsFromFile("..\creds\credFileHuman.txt")

# Uncomment - to make sure that creds are either set in code or read in correctly
#print("USERNAME="+str(USERNAME))
#print("PASSWORD="+str(PASSWORD))
#print("CLIENT_ID="+str(CLIENT_ID))

### Define Token Handling
​
Having a valid token is a pre-requisite to requesting of any RDP content

In [19]:
TOKEN_ENDPOINT = RDP_BASE_URL + CATEGORY_URL + RDP_AUTH_VERSION + ENDPOINT_URL

def _requestNewToken(refreshToken):
    if refreshToken is None:
        tData = {
            "username": USERNAME,
            "password": PASSWORD,
            "grant_type": "password",
            "scope": SCOPE,
            "takeExclusiveSignOnControl": "true"
        };
    else:
        tData = {
            "refresh_token": refreshToken,
            "grant_type": "refresh_token",
        };

    # Make a REST call to get latest access token
    response = requests.post(
        TOKEN_ENDPOINT,
        headers = {
            "Accept": "application/json"
        },
        data = tData,
        auth = (
            CLIENT_ID,
            CLIENT_SECRET
        )
    )
    
    if response.status_code != 200:
        raise Exception("Failed to get access token {0} - {1}".format(response.status_code, response.text));

    # Return the new token
    return json.loads(response.text);

def saveToken(tknObject):
    tf = open(TOKEN_FILE, "w+");
    print("Saving the new token");
    # Append the expiry time to token
    tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10;
    # Store it in the file
    json.dump(tknObject, tf, indent=4)
    
def getToken():
    try:
        print("Reading the token from: " + TOKEN_FILE);
        # Read the token from a file
        tf = open(TOKEN_FILE, "r+")
        tknObject = json.load(tf);

        # Is access token valid
        if tknObject["expiry_tm"] > time.time():
            # return access token
            return tknObject["access_token"];

        print("Token expired, refreshing a new one...");
        tf.close();
        # Get a new token from refresh token
        tknObject = _requestNewToken(tknObject["refresh_token"]);

    except Exception as exp:
        print("Caught exception: " + str(exp))
        print("Getting a new token using Password Grant...");
        tknObject = _requestNewToken(None);

    # Persist this token for future queries
    saveToken(tknObject)
    print("Token is: " + tknObject["access_token"])
    # Return access token
    return tknObject["access_token"];

###  Test getToken

In [20]:
accessToken = getToken();
print("Have token now");

Reading the token from: token.txt
Caught exception: Expecting value: line 1 column 1 (char 0)
Getting a new token using Password Grant...
Saving the new token
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwiWC1OT2h0WHN3Qkw5WjhyZDVlQ0x5QldMYkJzQ1BxT1c2b1hRSFNXNjliaFpZbzhsa2Q5TmRueHV6LVlnSEx2X3F2eWlpYi1lUDdIWEJJTmpqVU1ULUJJOEdBUHlWVEgtbk1oc29lQ3c5SnJ1OWNOODFyVDNNTjNoWUpYSER1QzdNYkF3SmFzMVBBNWhGVi14aF93UldGaDFSOUFpRDNJZTFNYzhOMlY3bTI4UGpnRDlkSk91WXZTZjhlMm8tR21KTkdpYTNFYVY3VnJaMDdnaEdRbUllampaVGEyakYyMEN1SWJBRzNLb0FPMmFMNGVRYmg2a3hqdWtzMV9Ddlk2U2xabUF5SGJFQ3dBUEFmRXJFeTRWaFBwNVhEc0ZIWjVfc0RSTWwtSlBkcjZ6Qm9kSDVwaE4yZU9QUXVrUmZsM1poejdrMHlJeVRvc19sQ3RiX0RHN0E3c05zamRXOUJ6M3hXYU5mNUp2TzhQTU54SE5YZnZ0eXVpUVd6Q2wtcGlVUDJLbi1KU2VWd0lHMHRSMGRDNEpUWVBoN1EtM0UyN1RQWkpUVnpJQXBLSGptRllWalNtSVdQZzk1ZFJQWGc0aTB5b0JqNWI2RDVVSUR0YlJaX01VNzhjcl85RXdzb08zb1otZm1XaWcxMEhEc3lwVU5UdExaX0ROdGNXaDY5eGFK